# Payload Configuration Sweep
Similar to the spreadsheet from before.

The inputs to the sweep are the following:
* Payload configurations to try
* Assumed parameters (e.g. L/D, target speeds)
* Historical data (for aerostructural mass ratio)

We could pick different historical data to infer the AS ratio from. For example, we could do all aircraft instead of just Rassam's picks.

The code also supports swapping in custom sets of components for e.g. if our mass budget changes.


In [1]:
import pandas as pd
from IPython.display import display

from wyvern.data import RASSAM_CORRELATIONS
from wyvern.analysis.payload_sweep import sweep_payload_configs
from wyvern.analysis.parameters import AssumedParameters

params = AssumedParameters(lift_to_drag_ratio=10,
                           cruise_speed=12,
                           turn_speed=11,
                           propulsive_efficiency=0.5,
                           configuration_bonus=1.3,
                           short_takeoff=False,
                           stability_distance=0)

payload_configs = [(8, i, 4) for i in range(0, 7)]
results = sweep_payload_configs(payload_configs, params, RASSAM_CORRELATIONS)

with pd.option_context('display.precision', 3):
    display(results)

Total Fixed Mass: 448.15 g
AS Mass Ratio: 45.263%


payload_mass empty_mass  as_mass takeoff_mass cargo_units  \
8 0 4        259.2    1033.06   584.91      1292.26         480   
  1 4       305.13    1071.04   622.89      1376.17         530   
  2 4       351.06   1109.019  660.869     1460.079         580   
  3 4       396.99   1146.999  698.849     1543.989         630   
  4 4       442.92   1184.979  736.829     1627.899         680   
  5 4       488.85   1222.958  774.808     1711.808         730   
  6 4       534.78   1260.938  812.788     1795.718         780   

      payload_fraction reached_pf_cap cargo_score_times_pf total_energy  \
8 0 4            0.201          False               15.106     2456.317   
  1 4            0.222          False               17.898     2615.812   
  2 4             0.24          False               20.673     2775.306   
  3 4            0.257           True               22.776     2934.801   
  4 4            0.272           True               24.027     3094.296   
  5 4            0.286           True                25.25     3253.791   
  6 4            0.298           True               26.449     3413.285   

      efficiency_score takeoff_bonus configuration_bonus stability_bonus  \
8 0 4            1.697           1.0                 1.3             1.0   
  1 4            1.497           1.0                 1.3             1.0   
  2 4            1.329           1.0                 1.3             1.0   
  3 4            1.189           1.0                 1.3             1.0   
  4 4            1.069           1.0                 1.3             1.0   
  5 4            0.967           1.0                 1.3             1.0   
  6 4            0.879           1.0                 1.3             1.0   

      total_flight_score  
8 0 4             33.329  
  1 4             34.821  
  2 4             35.729  
  3 4             35.202  
  4 4             33.405  
  5 4             31.749  
  6 4             30.221

## Additional Math

### Energy
Assumptions:
* Constant L/D
* Constant speed steady level flight and level turn, with no transition time between the two

In steady level flight,
$$ L = W $$
$$ D = \frac{L}{L/D} $$
$$ T = D$$
$$ P = T v = \frac{W}{L/D} v $$
$$ E_{\text{SLF}} = \frac{W}{L/D} v_{\text{SLF}} t_{\text{SLF}} $$

In a turn, the lift is increased by the load factor. The load factor is given by:
$$ n = \sqrt{\left(\frac{v^2}{g R}\right)^2 + 1}$$
$$ L = W n $$
and similarly as before,
$$ E_{\text{SLT}} = \frac{nW}{L/D} v_{\text{SLT}} t_{\text{SLT}} $$

To course is split into straights of length $\ell_s$ and turns of length $\ell_t$. The time spent in each is given by:
$$ t_{\text{SLF}} = \frac{\ell_s}{v_{\text{SLF}}} $$
$$ t_{\text{SLT}} = \frac{\ell_t}{v_{\text{SLT}}} $$

and the total energy is given by:
$$ E = \frac{W}{L/D} v_{\text{SLF}} \frac{\ell_s}{v_{\text{SLF}}} + \frac{nW}{L/D} v_{\text{SLT}} \frac{\ell_t}{v_{\text{SLT}}} $$
$$ E = \frac{W}{L/D} \ell_s + \frac{nW}{L/D} \ell_t $$
$$ E = \frac{W}{L/D} \left(\ell_s + n \ell_t\right)$$

and we will put the propulsive efficiency $\eta$ in the denominator:
$$ \boxed{E = \frac{W}{\eta L/D} \left(\ell_s + n \ell_t\right)} $$

which gives an underestimate of the energy required. The actual energy will be greater due to transients and other losses. Purportedly, 3300 J is a "very good" performance.